In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('../Dataset/cardekho_imputated.csv')

In [24]:
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [25]:
df.shape

(15411, 14)

In [26]:
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [27]:
df.drop('car_name',axis=1,inplace=True)
df.drop('brand',axis=1,inplace=True)

In [28]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [29]:
num_features = [feature for feature in df.columns if df[feature].dtype != 'O']
print('Num of numerical features: ',len(num_features))
cat_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print('Num of categorical features: ',len(cat_features))
dis_features = [feature for feature in num_features if len(df[feature].unique())<=25]
print('Num of discrete features: ',len(dis_features))
cont_features = [feature for feature in num_features if not dis_features]
print('Num of continuous feature: ',len(cont_features))

Num of numerical features:  8
Num of categorical features:  4
Num of discrete features:  2
Num of continuous feature:  0


In [30]:
from sklearn.model_selection import train_test_split
X = df.drop('selling_price',axis=1)
y = df['selling_price']

In [31]:
X.head()

,Unnamed: 0,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [32]:
y

0         120000
1         550000
2         215000
3         226000
4         570000
          ...   
15406     250000
15407     925000
15408     425000
15409    1225000
15410    1200000
Name: selling_price, Length: 15411, dtype: int64

In [33]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

In [34]:
num_features = X.select_dtypes(exclude='object').columns
oh_columns = ['seller_type','fuel_type','transmission_type']
label_encoder_columns = ['model']

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder',oh_transformer,oh_columns),
        ('StandardScaler',numeric_transformer,num_features)
    ],remainder='passthrough'
)

In [35]:
X = preprocessor.fit_transform(X)

In [36]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,y_train.shape

((12328, 15), (12328,))

In [37]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [38]:
def evaluate_model(true,predicted):
  mae = mean_absolute_error(true,predicted)
  mse = mean_squared_error(true,predicted)
  rmse =np.sqrt(mean_absolute_error(true,predicted))
  r2_square = r2_score(true,predicted)
  return mae, mse, rmse, r2_square

In [39]:
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'K-Neighbors Regressor': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Gradient Boost Regressor': GradientBoostingRegressor(),
    'XGBoost Regressor': XGBRegressor()
}

for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(X_train,y_train)

  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)

  model_train_mae, model_train_mse, model_train_rmse, model_train_r2_square = evaluate_model(y_train,y_train_pred)
  model_test_mae, model_test_mse, model_test_rmse, model_test_r2_square = evaluate_model(y_test,y_test_pred)

  print(list(models.keys())[i])

  print('Model Performance for Training Data')
  print('Mean Absolute Error: {:.4f}'.format(model_train_mae))
  print('Mean Squared Error: {:.4f}'.format(model_train_mse))
  print('Root Mean Squared Error: {:.4f}'.format(model_train_rmse))
  print('R Square: {:.4f}'.format(model_train_r2_square))

  print('----------------------------------------')

  print('Model Performance for Test Data')
  print('Mean Absolute Error: {:.4f}'.format(model_test_mae))
  print('Mean Squared Error: {:.4f}'.format(model_test_mse))
  print('Root Mean Squared Error: {:.4f}'.format(model_test_rmse))
  print('R Square: {:.4f}'.format(model_test_r2_square))

  print('='*35)

Linear Regression
Model Performance for Training Data
Mean Absolute Error: 268104.1303
Mean Squared Error: 306749877172.0585
Root Mean Squared Error: 517.7877
R Square: 0.6218
----------------------------------------
Model Performance for Test Data
Mean Absolute Error: 279686.6479
Mean Squared Error: 252588750547.2280
Root Mean Squared Error: 528.8541
R Square: 0.6645
Lasso
Model Performance for Training Data
Mean Absolute Error: 268101.7491
Mean Squared Error: 306749882060.9022
Root Mean Squared Error: 517.7854
R Square: 0.6218
----------------------------------------
Model Performance for Test Data
Mean Absolute Error: 279682.7929
Mean Squared Error: 252587811682.4362
Root Mean Squared Error: 528.8504
R Square: 0.6645
Ridge
Model Performance for Training Data
Mean Absolute Error: 268061.4421
Mean Squared Error: 306750591340.8743
Root Mean Squared Error: 517.7465
R Square: 0.6218
----------------------------------------
Model Performance for Test Data
Mean Absolute Error: 279625.1576


In [44]:
#Hyperparameter Tuning
xgr_param = {
               'n_estimators':[100,200,300],
               'max_depth':[5,8,15,None,10],
               'learning_rate': [0.1,0.01],
               'colsample_bytree': [0.5, 0.8, 1, 0.3, 0.4]}

In [45]:
randomcv_models = [('XGR',XGBRegressor(),xgr_param)]

In [46]:
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm

model_param = {}
for name, model, params in tqdm(randomcv_models, desc="Model Search Progress", unit="model"):
    random = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=100, cv=3, verbose=2, n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f'---------- Best Params for {model_name} ----------')
    print(model_param[model_name])

Model Search Progress:   0%|          | 0/1 [00:00<?, ?model/s]

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100; total time=   0.1s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=5, n_estimators=200; total time=   0.2s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=5, n_estimators=200; total time=   0.2s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100; total time=   0.1s
[CV] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100; total time=   0.1s
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=5, n_estimators=200; total time=   0.2s
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=5, n_estimators=200; total time=   0.1s
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=5, n_estimators=200; total time=   0.2s
[CV] END colsample_bytree=1, learning_rate=0.01, max_depth=8, n_estimators=200; total time=   0.8s
[CV] END colsample_bytree=1, learning_rat

Model Search Progress: 100%|██████████| 1/1 [00:28<00:00, 28.20s/model]

---------- Best Params for XGR ----------
{'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


In [49]:
models = {
    'Xgboost Regressor': XGBRegressor(n_estimators=300,max_depth=8,learning_rate=0.1,colsample_bytree=0.5)
}

for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(X_train,y_train)

  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)

  model_train_mae, model_train_mse, model_train_rmse, model_train_r2_square = evaluate_model(y_train,y_train_pred)
  model_test_mae, model_test_mse, model_test_rmse, model_test_r2_square = evaluate_model(y_test,y_test_pred)

  print(list(models.keys())[i])

  print('Model Performance for Training Data')
  print('Mean Absolute Error: {:.4f}'.format(model_train_mae))
  print('Mean Squared Error: {:.4f}'.format(model_train_mse))
  print('Root Mean Squared Error: {:.4f}'.format(model_train_rmse))
  print('R Square: {:.4f}'.format(model_train_r2_square))

  print('----------------------------------------')

  print('Model Performance for Test Data')
  print('Mean Absolute Error: {:.4f}'.format(model_test_mae))
  print('Mean Squared Error: {:.4f}'.format(model_test_mse))
  print('Root Mean Squared Error: {:.4f}'.format(model_test_rmse))
  print('R Square: {:.4f}'.format(model_test_r2_square))

  print('='*35)

Xgboost Regressor
Model Performance for Training Data
Mean Absolute Error: 37026.1766
Mean Squared Error: 2571652857.4143
Root Mean Squared Error: 192.4219
R Square: 0.9968
----------------------------------------
Model Performance for Test Data
Mean Absolute Error: 93696.0907
Mean Squared Error: 59812600356.4887
Root Mean Squared Error: 306.0982
R Square: 0.9205
